This is a quick dirty code to convert connectivity matrices to a tidy tabular format that can be used in stat tools.

In [43]:
from typing import Literal
from src.acnets.pipeline import Parcellation, ConnectivityExtractor
from pathlib import Path
import pandas as pd
from sklearn.pipeline import Pipeline
from src.acnets.pipeline.multiscale_classifier import ExtractH2Features
from src.acnets.pipeline.connectivity_aggregator import ConnectivityAggregator

kind = 'correlation'
atlas = 'dosenbach2010'
normalize = True
conn_subset: Literal['between', 'within'] = 'within'

pipe = Pipeline([
    ('parcellation', Parcellation(atlas_name=atlas, normalize=normalize)),
    ('extract_connectivity', ConnectivityExtractor(kind=kind)),
    ('aggregate_connectivity', ConnectivityAggregator(strategy='network')),
    ('h2_features', ExtractH2Features(subset=conn_subset))
])

features = pipe.fit_transform(None)

timeseries_ds = pipe.named_steps['parcellation'].fit_transform(None)

df = pd.DataFrame(
    features,
    columns=pipe.get_feature_names_out(),
    index=timeseries_ds['subject'],
)
df.index.name = 'subject'
df.reset_index(inplace=True)

df = df.melt(id_vars='subject', var_name='name', value_name='connectivity')

if conn_subset == 'between':
    df[['network_src', 'network_dst']] = df['name'].str.split(' \N{left right arrow} ', expand=True)
    df.drop(columns='name', inplace=True)

    df.to_csv('data/julia2018/derivatives/connectivity/'
            f'{atlas}_{kind.replace(" ","_")}_cagg-{conn_subset}.csv',
            index=False)
elif conn_subset == 'within':
    df.rename(columns={'name': 'network'}, inplace=True)
    df.to_csv('data/julia2018/derivatives/connectivity/'
            f'{atlas}/{kind.replace(" ","")}_cagg-{conn_subset}.csv',
            index=False)
df

,subject,network,connectivity
0,AVGP01,CER,0.441328
1,AVGP02,CER,0.336553
2,AVGP03,CER,0.462330
3,AVGP04,CER,0.234237
4,AVGP05,CER,0.354061
...,...,...,...
187,NVGP16,VIS,0.280626
188,NVGP17,VIS,0.420112
189,NVGP17NEW,VIS,0.239883
190,NVGP19,VIS,0.414607


In [ ]:


ce = ConnectivityExtractor(kind=kind)

ce_ds = ce.fit_transform(p.fit_transform(X=None))

df = ce_ds['connectivity'].to_dataframe().reset_index()

networks_df = ce_ds['network'].to_dataframe().reset_index()

df = df.merge(networks_df, left_on='region_src', right_on='region', how='left').rename(columns={'network': 'network_src'})
df = df.merge(networks_df, left_on='region_dst', right_on='region', how='left').rename(columns={'network': 'network_dst'})

df.drop(columns=['region_x', 'region_y'], inplace=True)
df = df[['subject', 'network_src', 'network_dst', 'region_src', 'region_dst', 'connectivity']]

df.to_csv('data/julia2018/derivatives/connectivity/dosenbach2010_{}.csv'.format(kind.replace(' ', '_')), index=False)